In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
from src.utils import setup_folders
from src.visualization.evaluate import calculate_metric_horizons, calculate_metric_horizon_windows, calculate_metric_horizons_all_models
from src.visualization.visualize import plot_horizons, plot_scatterplot, plot_test_day, plot_mean_std_error_multiple_models
from src.features.build_features import build_features, build_features_LSTM

pd.options.plotting.backend = "plotly"
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import n_colors

In [2]:
plant = "HPP1"
tech = "agr"
if plant in ["Nazeerabad", "HPP1", "HPP2", "HPP3"]:
    wind_cap = 1
    solar_cap = 1
agr_cap = wind_cap + solar_cap

In [3]:
# We load the predictions
filename = f'../../reports/{plant}_{tech}_predictions.csv'
df_preds_agr = pd.read_csv(filename, sep=";", index_col = [0], parse_dates=[0])

FileNotFoundError: [Errno 2] No such file or directory: '../../reports/HPP1_agr_predictions.csv'

In [ ]:
tech = "wind"
filename = f'../../reports/{plant}_{tech}_predictions.csv'
df_preds_wind = pd.read_csv(filename, sep=";", index_col = [0], parse_dates=[0])
tech = "solar"
filename = f'../../reports/{plant}_{tech}_predictions.csv'
df_preds_solar = pd.read_csv(filename, sep=";", index_col = [0], parse_dates=[0])

In [ ]:
# I am not sure about this calculation for now
df_preds_agr_sum = (df_preds_wind*wind_cap + df_preds_solar*solar_cap) / (agr_cap)
df_preds_agr_sum['Horizon'] = df_preds_agr['Horizon']

In [ ]:
df_preds_agr.columns

# Monte Carlo simulation

I have simulated the forecast errors for each horizon based on the standard deviation of the errors, where we make n = 10000 draws from a standard normal distribution. This makes the assumption that the error distributions are normal, however in the previous ridge plot we found that they were close to normal so we will use this assumption for now.

In [ ]:
def simulate_forecast_error_monte_carlo(df_preds, model, n_sim):
    
    df_sim = df_preds[[model, 'Y_true', 'Horizon']]
    df_sim.loc[:,'residuals'] = df_sim[model] - df_sim['Y_true']
    
    # We extract the quantiles [10, 25, 50, 75, 90]
    dict_quantiles = {0.01: np.array([]),
                      0.05: np.array([]),
                      0.1: np.array([]), 
                      0.25: np.array([]), 
                      0.75: np.array([]), 
                      0.9: np.array([]),
                      0.95: np.array([]),
                      0.99: np.array([])}
    
    dict_sim = {}
    for horizon in np.unique(df_sim['Horizon']):
        forecast_errors_horizon = df_sim.loc[df_sim['Horizon'] == horizon, 'residuals'].dropna().values
        std_forecast_errors_horizon = np.std(forecast_errors_horizon)
        
        # We simulate using a standard normal distribution N(0,1) such that we assume forecast errors are normal distributed
        sim_forecast_errors_horizon = np.random.normal(0, 1, n_sim) * std_forecast_errors_horizon
    
        # We save the result in a dictionary
        dict_sim[horizon] = sim_forecast_errors_horizon
        
        # We grab the quantiles
        for quantile in dict_quantiles.keys():
            dict_quantiles[quantile] = np.append(dict_quantiles[quantile], np.quantile(dict_sim[horizon], quantile))
            
    df_quantiles = pd.DataFrame(dict_quantiles, index = np.unique(df_sim['Horizon']))        
    
    return(df_quantiles)

In [ ]:
def plot_forecast_error_monte_carlo(df, model, n_sim, idx):
    forecast_error_quantiles = simulate_forecast_error_monte_carlo(df_preds_agr, model, n_sim)
    
    # We grab the day we want to plot
    df = df[df.columns.drop(['Horizon'])]
    df = df.dropna()

    forecast_days = df.index[df.index.time == datetime.time(0, 0)].strftime('%Y-%m-%d')
    forecast_day = forecast_days[idx]

    df_plot = df.loc[forecast_day, [model, 'Y_true']]
    
    # We update the index of the quantiles to match the datetimes
    forecast_error_quantiles.index = df_plot.index
    
    # Color choices
    n_bands = int(len(forecast_error_quantiles.columns)/2)
    fill_colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', n_bands, colortype='rgb')
    line_colors = px.colors.qualitative.Plotly
    
    fig = go.Figure()
    keys = np.array(forecast_error_quantiles.columns)
    for cnt in range(0, n_bands):
        color = fill_colors[cnt]

        # We plot the bands
        fig.add_trace(go.Scatter(x=df_plot.index,
                                y=df_plot[model]+forecast_error_quantiles[keys[cnt]],
                                mode='lines', 
                                showlegend = False,
                                line=dict(width=0, color=color)))
        fig.add_trace(go.Scatter(x=df_plot.index,
                                y=df_plot[model]+forecast_error_quantiles[keys[-(cnt+1)]],
                                mode='lines', 
                                fill='tonexty', # fill area between trace0 and trace1
                                name=f'{keys[-(cnt+1)]*100} quantile',
                                line=dict(width=0, color=color)))

    # We plot the deterministic part of the forecast
    fig.add_trace(go.Scatter(x=df_plot.index, 
                            y=df_plot[model],
                            mode='lines+markers',
                            name=f'{model}',
                            line_color='black'))
    fig.add_trace(go.Scatter(x=df_plot.index, 
                            y=df_plot['Y_true'],
                            mode='lines+markers',
                            name='Y_true',
                            line_color=line_colors[1]))
    
    return(fig)

In [ ]:
df_preds_agr

In [ ]:
plot_forecast_error_monte_carlo(df=df_preds_agr, model='Y_pred_LSTM(full day)', n_sim=10000, idx=21)

In [ ]:
plot_forecast_error_monte_carlo(df=df_preds_agr_sum, model='Y_pred_LSTM(full day)', n_sim=10000, idx=21)

In [ ]:
plot_forecast_error_monte_carlo(df=df_preds_solar, model='Y_pred_LSTM(full day)', n_sim=10000, idx=21)

In [ ]:
plot_forecast_error_monte_carlo(df=df_preds_wind, model='Y_pred_LSTM(full day)', n_sim=10000, idx=21)